
# EXPLORATORY ESPOL DATA ANALYSIS












## DATA SETS






Primero importamos todas las librerías que vamos a utilizar, posteriormente visualizaciones como esta las datas.

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import os
import plotly.express as px
import cufflinks as cf
from IPython.display import display,HTML

cf.set_config_file(sharing="public",theme="ggplot", offline=True)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
#df_H Horario_aula_20230302 is about classroom timetable  PAE
#df-PE Rpt PlanifHorarioExam_20230120  SEMESTRE
"""data_folder = '/content/drive/MyDrive/EDA_ Group_Slytherin /EDA_ESPOL'
df_H_p = pd.read_csv(os.path.join(data_folder,'data.csv'))
df_PE_s = pd.read_excel(os.path.join(data_folder,'Rpt PlanifHorarioExam_20230120.xlsx'),sheet_name='Sheet1')"""
df_H_p = pd.read_csv('data.csv')
df_PE_s = pd.read_excel('Rpt PlanifHorarioExam_20230120.xlsx',sheet_name='Sheet1')

In [ ]:
df_PE_s

,ANIO,TERMINO,FECHA,EXAMEN,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,...,ESELEARNING,TIPOELEARNING,ESHIBRIDO,MODALIDAD,UNIDAD,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS
0,2022,2S,2022-11-19,2,sábado,1970-01-01 08:00:00,1970-01-01 12:00:00,QUIMIOMETRIA,ESTP1033,1,...,NaN,NaN,NaN,NaN,Facultad de Ciencias Naturales y Matemáticas,NAME001,EMAIL001,A001,9M,CAMPUS GUSTAVO GALINDO
1,2022,2S,2022-11-26,M,sábado,1970-01-01 08:00:00,1970-01-01 12:00:00,QUIMIOMETRIA,ESTP1033,1,...,NaN,NaN,NaN,NaN,Facultad de Ciencias Naturales y Matemáticas,NAME001,EMAIL001,A001,9M,CAMPUS GUSTAVO GALINDO
2,2022,2S,2022-11-20,2,domingo,1970-01-01 08:00:00,1970-01-01 10:00:00,DISEÑO DE CIMENTACIONES SUPERFICIALES Y PROFUNDAS,ARQP1014,1,...,NaN,NaN,N,NaN,Facultad de Ingeniería en Ciencias de la Tierra,NAME002,EMAIL002,A001,9M,CAMPUS GUSTAVO GALINDO
3,2022,2S,2022-11-27,M,domingo,1970-01-01 08:00:00,1970-01-01 10:00:00,DISEÑO DE CIMENTACIONES SUPERFICIALES Y PROFUNDAS,ARQP1014,1,...,NaN,NaN,N,NaN,Facultad de Ingeniería en Ciencias de la Tierra,NAME002,EMAIL002,A001,9M,CAMPUS GUSTAVO GALINDO
4,2022,2S,2022-11-22,1,martes,1970-01-01 14:00:00,1970-01-01 16:00:00,INTRODUCCIÓN A LA INDUSTRIA HIDROCARBURÍFERA,PETG1032,1,...,N,NaN,N,PR,Facultad de Ingeniería en Ciencias de la Tierra,NAME003,EMAIL003,A102,13H,CAMPUS GUSTAVO GALINDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,2022,2S,2023-01-27,2,domingo,1970-01-01 13:00:00,1970-01-01 15:00:00,INVESTIGACIÓN I,CCAG1003,636,...,NaN,NaN,NaN,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,NAME347,EMAIL347,B-101 (AULA-EDCOM),AULAS EDCOM,CAMPUS GUSTAVO GALINDO
4132,2022,2S,2023-01-27,2,domingo,1970-01-01 13:00:00,1970-01-01 15:00:00,INVESTIGACIÓN I,CCAG1003,637,...,NaN,NaN,NaN,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,NAME475,EMAIL475,B-101 (AULA-EDCOM),AULAS EDCOM,CAMPUS GUSTAVO GALINDO
4133,2022,2S,2023-01-27,2,domingo,1970-01-01 13:00:00,1970-01-01 15:00:00,INVESTIGACIÓN I,CCAG1003,638,...,NaN,NaN,NaN,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,NAME097,EMAIL097,B-101 (AULA-EDCOM),AULAS EDCOM,CAMPUS GUSTAVO GALINDO
4134,2022,2S,2023-01-27,2,domingo,1970-01-01 13:00:00,1970-01-01 15:00:00,INVESTIGACIÓN I,CCAG1003,639,...,NaN,NaN,NaN,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,NAME547,EMAIL547,B-101 (AULA-EDCOM),AULAS EDCOM,CAMPUS GUSTAVO GALINDO


In [ ]:
df_H_p

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS
0,2023,0S,miércoles,9:00:00 a. m.,11:00:00 a. m.,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
1,2023,0S,lunes,9:00:00 a. m.,12:00:00 p. m.,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
2,2023,0S,miércoles,2:00:00 p. m.,4:00:00 p. m.,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
3,2023,0S,lunes,2:00:00 p. m.,5:00:00 p. m.,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
4,2023,0S,lunes,9:00:00 a. m.,12:00:00 p. m.,TÉCNICAS DIETÉTICAS,NUTG2042,"1,00",N,"20,00","20,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,2:00:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
123,2023,0S,martes,3:30:00 p. m.,5:00:00 p. m.,INGLÉS IV,IDIG1009,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO
124,2023,0S,martes,3:30:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
125,2023,0S,miércoles,2:00:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"105,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO


##  DATA SET HORARIOS PAE 2023



Con esto podemos ver que materias se  pueden recibir en el PAE, a comparación de otras que solo se puede ver en semestre (PAO)

In [ ]:
subjects_all = df_PE_s["NOMBRE"].unique()
subjects_pae = df_H_p["NOMBRE"].unique()
subjects_no_pae = subjects_all[np.isin(subjects_all,subjects_pae)==False] 

In [ ]:
df_materias = df_PE_s.drop_duplicates(subset=['NOMBRE'])[['NOMBRE','UNIDAD']]
df_materias['CLASIFICACION'] = ['PAO']*df_materias.shape[0]
df_materias['CLASIFICACION'][np.isin(subjects_all,subjects_pae)]='PAE'
df_materias

,NOMBRE,UNIDAD,CLASIFICACION
0,QUIMIOMETRIA,Facultad de Ciencias Naturales y Matemáticas,PAO
2,DISEÑO DE CIMENTACIONES SUPERFICIALES Y PROFUNDAS,Facultad de Ingeniería en Ciencias de la Tierra,PAO
4,INTRODUCCIÓN A LA INDUSTRIA HIDROCARBURÍFERA,Facultad de Ingeniería en Ciencias de la Tierra,PAO
5,ESTRATIGRAFIA Y SEDIMENTACIÓN,Facultad de Ingeniería en Ciencias de la Tierra,PAO
10,PATOLOGÍA ACUÍCOLA I,Facultad de Ingeniería Marítima y Ciencias del...,PAO
...,...,...,...
2628,DISEÑO ESTRATÉGICO,"Facultad de Arte, Diseño y Comunicación Audiov...",PAO
2660,CÁLCULO DE VARIAS VARIABLES,Facultad de Ciencias Naturales y Matemáticas,PAO
2693,FÍSICA: TERMODINÁMICA Y ELECTRÓNICA,Facultad de Ciencias Naturales y Matemáticas,PAO
4061,INVESTIGACIÓN II,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,PAO


Idea donde este todas.

In [ ]:
import plotly.express as px
fig = px.treemap(df_materias, path=[px.Constant("Todas las materias"), 'UNIDAD', 'NOMBRE', 'CLASIFICACION'],height=,
                color_discrete_map={'PAE':'darkblue'})
fig.show()

SyntaxError: invalid syntax (811621694.py, line 2)

Idea por facultad 

In [ ]:
for fac in df_materias.UNIDAD:
    df_materias_fac = df_materias[df_materias.UNIDAD==fac]
    fig = px.treemap(df_materias_fac, path=[px.Constant(f"{fac}"), 'NOMBRE', 'CLASIFICACION'],
                color_discrete_map={'PAE':'darkblue', 'PAO':'red'})
    fig.show()

SyntaxError: invalid syntax (215350180.py, line 3)

In [ ]:
import plotly.express as px
fig = px.treemap(df_materias,path=[df_materias.index],height=1000)
fig.update_traces(root_color="lightgrey")
fig.show()

In [ ]:
fig_m = px.bar(df_materias,x="DISPONIBLEPAE",color='CLASIFICACION',title ="MATERIAS DISPONIBLES EN EL PAE",height=40000,
               color_discrete_map={ # replaces default color 
                "PAO": "red", "PAE": "green"
            })
fig_m.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['NOMBRE', 'UNIDAD', 'CLASIFICACION'] but received: DISPONIBLEPAE

In [ ]:
subjects_pae

array(['PRODUCCIÓN VEGETAL I', 'DIAGNÓSTICO Y PROTECCIÓN VEGETAL I',
       'TÉCNICAS DIETÉTICAS', 'CÁLCULO DE UNA VARIABLE',
       'CÁLCULO VECTORIAL', 'SISTEMAS DIGITALES I',
       'PRINCIPIOS DE ELECTRÓNICA', 'ÁLGEBRA LINEAL', 'ESTADÍSTICA',
       'INGLÉS III', 'INGLÉS IV', 'INGLÉS I', 'INGLÉS II', 'INGLÉS V',
       'CIENCIAS DE LA SOSTENIBILIDAD', 'FUNDAMENTOS DE PROGRAMACIÓN',
       'EMPRENDIMIENTO E INNOVACIÓN'], dtype=object)

In [ ]:
df_H_p.dtypes

ANIO                int64
TERMINO            object
DIA                object
HORAINICIO         object
HORAFIN            object
NOMBRE             object
CODIGOMATERIA      object
PARALELO           object
ESQUINCENA         object
CAPACIDAD          object
CUPOPLANIFICADO    object
MODALIDAD          object
UNIDAD             object
TIPOMATERIA        object
DOCENTE            object
EMAIL              object
AULA               object
BLOQUE             object
CAMPUS             object
dtype: object

In [ ]:
df_H_p.head()

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS
0,2023,0S,miércoles,9:00:00 a. m.,11:00:00 a. m.,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
1,2023,0S,lunes,9:00:00 a. m.,12:00:00 p. m.,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
2,2023,0S,miércoles,2:00:00 p. m.,4:00:00 p. m.,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
3,2023,0S,lunes,2:00:00 p. m.,5:00:00 p. m.,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
4,2023,0S,lunes,9:00:00 a. m.,12:00:00 p. m.,TÉCNICAS DIETÉTICAS,NUTG2042,"1,00",N,"20,00","20,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO


In [ ]:
df_H_p.tail()


,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS
122,2023,0S,miércoles,2:00:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
123,2023,0S,martes,3:30:00 p. m.,5:00:00 p. m.,INGLÉS IV,IDIG1009,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO
124,2023,0S,martes,3:30:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
125,2023,0S,miércoles,2:00:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"105,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO
126,2023,0S,martes,3:30:00 p. m.,5:00:00 p. m.,INGLÉS V,IDIG1010,"105,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO


In [ ]:
def convert_time(df,feature):
    li = []
    for x in list(df[feature]):
        li_ho = x.split(' ')
        if 'a' in li_ho[1]:
            li_t = li_ho[0].split(':')
            if len(li_t[0]) == 1:
                n = '0'+li_ho[0] + 'AM'
                li.append(n)
            elif len(li_t[0]) > 1:
                n = li_ho[0] + 'AM'
                li.append(n)

        elif 'p' in li_ho[1]:
            li_t = li_ho[0].split(':')
            if 1 == len(li_t[0]):
                n = '0' + li_ho[0] + 'PM'
                li.append(n)
            elif len(li_t[0]) > 1:
                n = li_ho[0] + 'PM'
                li.append(n)

    df[feature] = li

In [ ]:
convert_time(df_H_p,'HORAINICIO')
convert_time(df_H_p,'HORAFIN')

In [ ]:
df_H_p

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS
0,2023,0S,miércoles,09:00:00AM,11:00:00AM,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
1,2023,0S,lunes,09:00:00AM,12:00:00PM,PRODUCCIÓN VEGETAL I,AGRG1032,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO
2,2023,0S,miércoles,02:00:00PM,04:00:00PM,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
3,2023,0S,lunes,02:00:00PM,05:00:00PM,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,"1,00",N,"5,00","5,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO
4,2023,0S,lunes,09:00:00AM,12:00:00PM,TÉCNICAS DIETÉTICAS,NUTG2042,"1,00",N,"20,00","20,00",PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,02:00:00PM,05:00:00PM,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
123,2023,0S,martes,03:30:00PM,05:00:00PM,INGLÉS IV,IDIG1009,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO
124,2023,0S,martes,03:30:00PM,05:00:00PM,INGLÉS V,IDIG1010,"104,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO
125,2023,0S,miércoles,02:00:00PM,05:00:00PM,INGLÉS V,IDIG1010,"105,00",N,"30,00","30,00",PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO


In [ ]:
import datetime as dt
df_H_p['HORAINICIO'] = pd.to_datetime(df_H_p['HORAINICIO'],format="%I:%M:%S%p")
df_H_p['HORAFIN'] = pd.to_datetime(df_H_p['HORAFIN'],format="%I:%M:%S%p")
df_H_p['DURACION'] = (df_H_p['HORAFIN']-df_H_p['HORAINICIO'])
df_H_p['HORAINICIO'] = [dt.datetime.strptime(str(t),'%Y-%m-%d %H:%M:%S').time() for  t in df_H_p['HORAINICIO']]
df_H_p['HORAFIN'] = [dt.datetime.strptime(str(t),'%Y-%m-%d %H:%M:%S').time() for  t in df_H_p['HORAFIN']]
df_H_p['DURACION'] = [(str(d).split('s')[1].strip()) for d in df_H_p['DURACION']]

In [ ]:
def convert_int(df,name_column):
    df[name_column] = [(n.split(",")[0])for n in df[name_column]]
    df[name_column] = df_H_p[name_column].astype(int)
    return df

for column in ['PARALELO','CAPACIDAD','CUPOPLANIFICADO']:
    convert_int(df_H_p,column)

In [ ]:
df_H_p

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION
0,2023,0S,miércoles,09:00:00,11:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00
1,2023,0S,lunes,09:00:00,12:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00
2,2023,0S,miércoles,14:00:00,16:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00
3,2023,0S,lunes,14:00:00,17:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00
4,2023,0S,lunes,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,03:00:00
123,2023,0S,martes,15:30:00,17:00:00,INGLÉS IV,IDIG1009,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO,01:30:00
124,2023,0S,martes,15:30:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,01:30:00
125,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,105,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO,03:00:00


In [ ]:
df_H_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ANIO             127 non-null    int64 
 1   TERMINO          127 non-null    object
 2   DIA              127 non-null    object
 3   HORAINICIO       127 non-null    object
 4   HORAFIN          127 non-null    object
 5   NOMBRE           127 non-null    object
 6   CODIGOMATERIA    127 non-null    object
 7   PARALELO         127 non-null    int64 
 8   ESQUINCENA       127 non-null    object
 9   CAPACIDAD        127 non-null    int64 
 10  CUPOPLANIFICADO  127 non-null    int64 
 11  MODALIDAD        127 non-null    object
 12  UNIDAD           127 non-null    object
 13  TIPOMATERIA      127 non-null    object
 14  DOCENTE          127 non-null    object
 15  EMAIL            127 non-null    object
 16  AULA             127 non-null    object
 17  BLOQUE           127 non-null    ob

In [ ]:
df_H_p['ESQUINCENA'].value_counts()

ESQUINCENA
N    127
Name: count, dtype: int64

In [ ]:
df_H_p['UNIDAD'].value_counts()

UNIDAD
Facultad de Ciencias Sociales y Humanísticas            68
Facultad de Ingeniería en Electricidad y Computación    22
Facultad de Ciencias Naturales y Matemáticas            21
Facultad de Ciencias de la Vida                         10
ESCUELA SUPERIOR POLITECNICA DEL LITORAL                 6
Name: count, dtype: int64

In [ ]:
df_H_p[['NOMBRE', 'AULA']].value_counts()

NOMBRE                              AULA 
CÁLCULO VECTORIAL                   A008     6
SISTEMAS DIGITALES I                L210     6
PRINCIPIOS DE ELECTRÓNICA           L002     6
CÁLCULO DE UNA VARIABLE             A001     6
INGLÉS IV                           A005     4
INGLÉS II                           A103     4
INGLÉS V                            A104     4
INGLÉS IV                           A104     4
                                    A103     4
                                    A004     4
INGLÉS III                          A101     4
                                    A002     4
                                    A001     4
INGLÉS II                           A101     4
                                    A102     4
INGLÉS V                            A102     4
INGLÉS I                            A003     4
                                    A001     4
FUNDAMENTOS DE PROGRAMACIÓN         L203     4
INGLÉS V                            A105     4
                  

In [ ]:
df_H_p[['AULA', 'UNIDAD']].value_counts()

AULA   UNIDAD                                              
A103   Facultad de Ciencias Sociales y Humanísticas            12
A001   Facultad de Ciencias Sociales y Humanísticas            10
A101   Facultad de Ciencias Sociales y Humanísticas             8
A104   Facultad de Ciencias Sociales y Humanísticas             8
A102   Facultad de Ciencias Sociales y Humanísticas             8
A001   Facultad de Ciencias Naturales y Matemáticas             6
L002   Facultad de Ingeniería en Electricidad y Computación     6
A104   Facultad de Ingeniería en Electricidad y Computación     6
L210   Facultad de Ingeniería en Electricidad y Computación     6
A008   Facultad de Ciencias Naturales y Matemáticas             6
A002   Facultad de Ciencias Sociales y Humanísticas             6
A005   Facultad de Ciencias Sociales y Humanísticas             4
A004   Facultad de Ciencias Sociales y Humanísticas             4
A003   Facultad de Ciencias Sociales y Humanísticas             4
A105   Facultad 

## HORARIO POR MATERIA


In [ ]:
df_H_p.NOMBRE.nunique()

17

In [ ]:
df_H_p.query("NOMBRE == 'TÉCNICAS DIETÉTICAS'")


,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION
4,2023,0S,lunes,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00
5,2023,0S,miércoles,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00
70,2023,0S,martes,09:00:00,11:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,100,N,10,10,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,L011,3K,CAMPUS GUSTAVO GALINDO,02:00:00
75,2023,0S,martes,14:00:00,17:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,101,N,10,10,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,L011,3K,CAMPUS GUSTAVO GALINDO,03:00:00


In [ ]:
cond=[df_H_p['PARALELO'] ==100,df_H_p['PARALELO'] ==101,df_H_p['PARALELO']==102,df_H_p['PARALELO']==103,df_H_p['PARALELO']==104,df_H_p['PARALELO']==105,df_H_p['PARALELO']==106]
result=[1,1,2,3,4,5,6]
df_H_p['COURSE']=np.select(cond,result,default=df_H_p['PARALELO'])
#solo tiene un paralelo
df_H_p['COURSE'][np.isin(df_H_p['NOMBRE'],['SISTEMAS DIGITALES I','PRINCIPIOS DE ELECTRÓNICA'])] = 1
cond=[df_H_p['PARALELO']>=100,df_H_p['PARALELO']<100]
result=['Práctico','Teórico']
df_H_p['CLASE']=np.select(cond,result)

La materia de TÉCNICAS DIETÉTICAS tiene un solo paparelo de teórico, pero dos de practico donde la mitad de los alumnos va a uno y la otra mitad al otro, fuente:  https://www.academico.espol.edu.ec/UI/Registros/MateriaPlanificada.aspx?idMateria=6973&numeroParalelo=1&p=587&idCurso=159345

In [ ]:
X = df_H_p[[ 'DIA','HORAINICIO','HORAFIN','PARALELO','NOMBRE','DOCENTE','COURSE','CLASE','CAPACIDAD','CUPOPLANIFICADO']][np.isin(df_H_p['NOMBRE'],['SISTEMAS DIGITALES I','PRINCIPIOS DE ELECTRÓNICA'])].groupby(['NOMBRE','DOCENTE','PARALELO','HORAINICIO','CAPACIDAD','CUPOPLANIFICADO',
                                                                                                                                                                                     'HORAFIN','DIA','CLASE']).count()
X
#Como podemos ver de un paralelo teórico , dividieron en 3 paralelos prácticos.

COURSE
NOMBRE                    DOCENTE PARALELO HORAINICIO CAPACIDAD CUPOPLANIFICADO HORAFIN  DIA       CLASE           
PRINCIPIOS DE ELECTRÓNICA NAME007 1        09:00:00   30        30              11:00:00 lunes     Teórico        1
                                                                                         martes    Teórico        1
                                                                                         miércoles Teórico        1
                                  101      09:00:00   10        10              10:00:00 jueves    Práctico       1
                                           11:00:00   10        10              13:00:00 lunes     Práctico       1
                          NAME031 102      10:00:00   10        10              11:00:00 jueves    Práctico       1
                                           11:00:00   10        10              13:00:00 martes    Práctico       1
                                  103      11:00:00   10        10              12:00:00 jueves    Práctico       1
                                                                                13:00:00 miércoles Práctico       1
SISTEMAS DIGITALES I      NAME006 1        09:00:00   30        30              11:00:00 lunes     Teórico        1
                                                                                         martes    Teórico        1
                                                                                         miércoles Teórico        1
                                  101      09:00:00   10        10              10:00:00 jueves    Práctico       1
                                           11:00:00   10        10              13:00:00 lunes     Práctico       1
                          NAME032 102      11:00:00   10        10              12:00:00 jueves    Práctico       1
                                                                                13:00:00 martes    Práctico       1
                                  103      11:00:00   10        10              13:00:00 miércoles Práctico       1
                                           12:00:00   10        10              13:00:00 jueves    Práctico       1

In [ ]:
df_H_p

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION,COURSE,CLASE
0,2023,0S,miércoles,09:00:00,11:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico
1,2023,0S,lunes,09:00:00,12:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico
2,2023,0S,miércoles,14:00:00,16:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico
3,2023,0S,lunes,14:00:00,17:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico
4,2023,0S,lunes,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,03:00:00,4,Práctico
123,2023,0S,martes,15:30:00,17:00:00,INGLÉS IV,IDIG1009,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico
124,2023,0S,martes,15:30:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico
125,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,105,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO,03:00:00,5,Práctico


In [ ]:
X = df_H_p[[ 'DIA','HORAINICIO','HORAFIN','PARALELO','NOMBRE','DOCENTE','COURSE','CLASE','CAPACIDAD','CUPOPLANIFICADO']][np.isin(df_H_p['NOMBRE'],['INGLÉS V'])].groupby(['NOMBRE','DOCENTE','PARALELO','HORAINICIO','CAPACIDAD','CUPOPLANIFICADO',
                                                                                                                                                                                     'HORAFIN','DIA','CLASE']).count()
X

COURSE
NOMBRE   DOCENTE PARALELO HORAINICIO CAPACIDAD CUPOPLANIFICADO HORAFIN  DIA       CLASE           
INGLÉS V NAME014 1        14:00:00   30        30              15:30:00 martes    Teórico        1
                                                               17:00:00 lunes     Teórico        1
                 101      14:00:00   30        30              17:00:00 miércoles Práctico       1
                          15:30:00   30        30              17:00:00 martes    Práctico       1
         NAME021 2        14:00:00   30        30              15:30:00 martes    Teórico        1
                                                               17:00:00 lunes     Teórico        1
                 102      14:00:00   30        30              17:00:00 miércoles Práctico       1
                          15:30:00   30        30              17:00:00 martes    Práctico       1
         NAME027 3        14:00:00   30        30              15:30:00 martes    Teórico        1
                                                               17:00:00 lunes     Teórico        1
                 103      14:00:00   30        30              17:00:00 miércoles Práctico       1
                          15:30:00   30        30              17:00:00 martes    Práctico       1
         NAME029 4        14:00:00   30        30              15:30:00 martes    Teórico        1
                                                               17:00:00 lunes     Teórico        1
                 104      14:00:00   30        30              17:00:00 miércoles Práctico       1
                          15:30:00   30        30              17:00:00 martes    Práctico       1
         NAME030 5        14:00:00   30        30              15:30:00 martes    Teórico        1
                                                               17:00:00 lunes     Teórico        1
                 105      14:00:00   30        30              17:00:00 miércoles Práctico       1
                          15:30:00   30        30              17:00:00 martes    Práctico       1

In [ ]:
df_H_p['NOMBRE'].value_counts()

NOMBRE
INGLÉS V                              20
INGLÉS IV                             16
INGLÉS III                            12
INGLÉS II                             12
SISTEMAS DIGITALES I                   9
PRINCIPIOS DE ELECTRÓNICA              9
INGLÉS I                               8
EMPRENDIMIENTO E INNOVACIÓN            6
CÁLCULO DE UNA VARIABLE                6
CÁLCULO VECTORIAL                      6
TÉCNICAS DIETÉTICAS                    4
FUNDAMENTOS DE PROGRAMACIÓN            4
CIENCIAS DE LA SOSTENIBILIDAD          3
PRODUCCIÓN VEGETAL I                   3
DIAGNÓSTICO Y PROTECCIÓN VEGETAL I     3
ÁLGEBRA LINEAL                         3
ESTADÍSTICA                            3
Name: count, dtype: int64

Crear data para gráfica

In [ ]:
df_Horario = pd.DataFrame(columns=['NOMBRE', 'HORARIO','INDICAR','PARALELO','DIA','COURSE','CLASE'])
df_Horario['NOMBRE']=list(df_H_p['NOMBRE'])*2
df_Horario['HORARIO']=list(df_H_p['HORAINICIO'])+list(df_H_p['HORAFIN'])
df_Horario['INDICAR'] = (['ENTRADA']*127)+(["SALIDA"]*127)
df_Horario['PARALELO'] = list(df_H_p['PARALELO'])*2
df_Horario['DIA']=list(df_H_p['DIA'])*2
df_Horario['COURSE'] = list(df_H_p['COURSE'])*2
df_Horario['CLASE'] = list(df_H_p['CLASE'])*2
dias_ordenados = ["lunes", "martes", "miércoles", "jueves"]
df_Horario.DIA = pd.Categorical(df_Horario['DIA'], dias_ordenados)
df_Horario = df_Horario.sort_values(by=['DIA','COURSE'])
df_Horario['size']=[10]*254
df_Horario

,NOMBRE,HORARIO,INDICAR,PARALELO,DIA,COURSE,CLASE,size
1,PRODUCCIÓN VEGETAL I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
3,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,14:00:00,ENTRADA,1,lunes,1,Teórico,10
4,TÉCNICAS DIETÉTICAS,09:00:00,ENTRADA,1,lunes,1,Teórico,10
10,SISTEMAS DIGITALES I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
13,PRINCIPIOS DE ELECTRÓNICA,09:00:00,ENTRADA,1,lunes,1,Teórico,10
...,...,...,...,...,...,...,...,...
207,CÁLCULO VECTORIAL,12:00:00,SALIDA,101,jueves,1,Práctico,10
221,PRINCIPIOS DE ELECTRÓNICA,11:00:00,SALIDA,102,jueves,1,Práctico,10
222,SISTEMAS DIGITALES I,12:00:00,SALIDA,102,jueves,1,Práctico,10
234,PRINCIPIOS DE ELECTRÓNICA,12:00:00,SALIDA,103,jueves,1,Práctico,10


In [ ]:
U = df_Horario.query("NOMBRE=='INGLÉS V'")
U[U['COURSE']==1]

,NOMBRE,HORARIO,INDICAR,PARALELO,DIA,COURSE,CLASE,size
33,INGLÉS V,14:00:00,ENTRADA,1,lunes,1,Teórico,10
160,INGLÉS V,17:00:00,SALIDA,1,lunes,1,Teórico,10
28,INGLÉS V,14:00:00,ENTRADA,1,martes,1,Teórico,10
91,INGLÉS V,15:30:00,ENTRADA,101,martes,1,Práctico,10
155,INGLÉS V,15:30:00,SALIDA,1,martes,1,Teórico,10
218,INGLÉS V,17:00:00,SALIDA,101,martes,1,Práctico,10
86,INGLÉS V,14:00:00,ENTRADA,101,miércoles,1,Práctico,10
213,INGLÉS V,17:00:00,SALIDA,101,miércoles,1,Práctico,10


In [ ]:
df_Horario[df_Horario['COURSE']==1]

,NOMBRE,HORARIO,INDICAR,PARALELO,DIA,COURSE,CLASE,size
1,PRODUCCIÓN VEGETAL I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
3,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,14:00:00,ENTRADA,1,lunes,1,Teórico,10
4,TÉCNICAS DIETÉTICAS,09:00:00,ENTRADA,1,lunes,1,Teórico,10
10,SISTEMAS DIGITALES I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
13,PRINCIPIOS DE ELECTRÓNICA,09:00:00,ENTRADA,1,lunes,1,Teórico,10
...,...,...,...,...,...,...,...,...
207,CÁLCULO VECTORIAL,12:00:00,SALIDA,101,jueves,1,Práctico,10
221,PRINCIPIOS DE ELECTRÓNICA,11:00:00,SALIDA,102,jueves,1,Práctico,10
222,SISTEMAS DIGITALES I,12:00:00,SALIDA,102,jueves,1,Práctico,10
234,PRINCIPIOS DE ELECTRÓNICA,12:00:00,SALIDA,103,jueves,1,Práctico,10


In [ ]:
import plotly.express as px
materias = list(df_H_p.NOMBRE.unique())
for mate in materias:
    df = df_Horario[df_Horario.NOMBRE == mate]
    l_course = df.COURSE.unique()
    for n_course in l_course:
        df_course = df[df.COURSE == n_course]
        fig = px.scatter(df_course, x="DIA", y="HORARIO",color='INDICAR',symbol='INDICAR',size='size',hover_data=['PARALELO','CLASE'],title=f'{mate} Course {n_course}')
        fig.show()


Podria borrar aqui

In [ ]:
df_Horario.DIA.value_counts()

DIA
martes       104
miércoles     66
lunes         64
jueves        20
Name: count, dtype: int64

In [ ]:
U = df_Horario.query("NOMBRE=='INGLÉS V'")
U

,NOMBRE,HORARIO,INDICAR,PARALELO,DIA,COURSE,CLASE,size
33,INGLÉS V,14:00:00,ENTRADA,1,lunes,1,Teórico,10
160,INGLÉS V,17:00:00,SALIDA,1,lunes,1,Teórico,10
49,INGLÉS V,14:00:00,ENTRADA,2,lunes,2,Teórico,10
176,INGLÉS V,17:00:00,SALIDA,2,lunes,2,Teórico,10
61,INGLÉS V,14:00:00,ENTRADA,3,lunes,3,Teórico,10
188,INGLÉS V,17:00:00,SALIDA,3,lunes,3,Teórico,10
65,INGLÉS V,14:00:00,ENTRADA,4,lunes,4,Teórico,10
192,INGLÉS V,17:00:00,SALIDA,4,lunes,4,Teórico,10
67,INGLÉS V,14:00:00,ENTRADA,5,lunes,5,Teórico,10
194,INGLÉS V,17:00:00,SALIDA,5,lunes,5,Teórico,10


In [ ]:
U = U.drop('PARALELO',axis=1)
t =  U.groupby(['COURSE']).get_group(1).value_counts()
t


NOMBRE    HORARIO   INDICAR  DIA        COURSE  CLASE     size
INGLÉS V  14:00:00  ENTRADA  lunes      1       Teórico   10      1
                             martes     1       Teórico   10      1
                             miércoles  1       Práctico  10      1
          15:30:00  ENTRADA  martes     1       Práctico  10      1
                    SALIDA   martes     1       Teórico   10      1
          17:00:00  SALIDA   lunes      1       Teórico   10      1
                             martes     1       Práctico  10      1
                             miércoles  1       Práctico  10      1
Name: count, dtype: int64

In [ ]:
list(df_Horario['DIA'].unique())

['lunes', 'martes', 'miércoles', 'jueves']

## MAPA

In [ ]:
pd.set_option('display.max_columns', 500)
#una materia 

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
dic_ubi={
    '9M':[-2.14662, -79.96666],
    '3H':[-2.1523, -79.95582],
    'GEA':[-2.14016, -79.96282],
    '13B':[-2.14558, -79.96507],
    '7B':[-2.14704, -79.96609] ,
    '11A':[-2.14466, -79.96772],
    '8F':[-2.14878, -79.96752],
    '8E':[-2.14873, -79.96782],
    '9F':[-2.14715, -79.96778],
    '11C':[-2.14563, -79.96683],
    '11D':[-2.14544, -79.96601],
    '3K':[-2.15122, -79.95424],
    
}

In [ ]:
df_H_p['BLOQUE']=[(i.strip())for i in df_H_p['BLOQUE']]
df_H_p['lat']=[(dic_ubi[bl][0])for bl in list(df_H_p['BLOQUE'])]
df_H_p['lon']=[(dic_ubi[bl][1])for bl in list(df_H_p['BLOQUE'])]

In [ ]:
espol_ubi = df_H_p[['NOMBRE','BLOQUE','CAPACIDAD','CLASE','lat','lon']].groupby(['NOMBRE', 'BLOQUE','CAPACIDAD','CLASE','lat','lon'], as_index= False).count()

In [ ]:
def normalizar_lon(df,bloque,lon):
    modiCor=[]
    compa=[]
    cont=0
    for bloque,lo in zip (df[bloque],df[lon]):
        if bloque not in compa:
            compa.append(bloque)
            modiCor.append(lo)
        else:
            cont=cont+0.000009
            
            modiCor.append(lo-cont)
    
    df[lon+'2']=modiCor

In [ ]:
def normalizar_lat(df,bloque,lat):
    modiCor=[]
    compa=[]
    cont=0
    for bloque,lo in zip (df[bloque],df[lat]):
        if bloque not in compa:
            compa.append(bloque)
            modiCor.append(lo)
        else:
            cont=cont+0.0000009
            
            modiCor.append(lo-cont)
    
    df[lat+'2']=modiCor

In [ ]:
normalizar_lon(espol_ubi,'BLOQUE','lon')
normalizar_lat(espol_ubi,'BLOQUE','lat')
espol_ubi

,NOMBRE,BLOQUE,CAPACIDAD,CLASE,lat,lon,lon2,lat2
0,CIENCIAS DE LA SOSTENIBILIDAD,9M,30,Teórico,-2.14662,-79.96666,-79.966660,-2.146620
1,CÁLCULO DE UNA VARIABLE,9M,30,Práctico,-2.14662,-79.96666,-79.966669,-2.146621
2,CÁLCULO DE UNA VARIABLE,9M,30,Teórico,-2.14662,-79.96666,-79.966678,-2.146622
3,CÁLCULO VECTORIAL,9M,30,Práctico,-2.14662,-79.96666,-79.966687,-2.146623
4,CÁLCULO VECTORIAL,9M,30,Teórico,-2.14662,-79.96666,-79.966696,-2.146624
5,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,3H,5,Teórico,-2.15230,-79.95582,-79.955820,-2.152300
6,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,GEA,5,Práctico,-2.14016,-79.96282,-79.962820,-2.140160
7,EMPRENDIMIENTO E INNOVACIÓN,13B,38,Práctico,-2.14558,-79.96507,-79.965070,-2.145580
8,EMPRENDIMIENTO E INNOVACIÓN,13B,38,Teórico,-2.14558,-79.96507,-79.965115,-2.145585
9,EMPRENDIMIENTO E INNOVACIÓN,7B,38,Práctico,-2.14704,-79.96609,-79.966090,-2.147040


In [ ]:
#px.set_mapbox_access_token('pk.eyJ1Ijoiam9udGh6IiwiYSI6ImNsZ3VkNnB4ZDJicW8zdGx1cXR5c21mcjAifQ.80mZH8cBu3LlgBN_0s8BKQ')
#se opto por usar el open street map porque esta mas bonito 

fig1 = px.scatter_mapbox(espol_ubi, lat="lat2", lon="lon2", text="NOMBRE", hover_data=["BLOQUE", "CAPACIDAD", 'CLASE'],
                         zoom=15, height=600,color_continuous_scale=px.colors.cyclical.IceFire,color='NOMBRE',size='CAPACIDAD')
fig1.update_layout(mapbox_style="open-street-map", mapbox=dict(
        center=dict(
            lat=-2.14756,
            lon=-79.9645
        ),))
fig1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig1.show()

## Cantidad de paralelos 

In [ ]:
b =df_H_p[["PARALELO",'CLASE','UNIDAD', 'BLOQUE','NOMBRE']].groupby(["PARALELO",'CLASE', 'UNIDAD', 'BLOQUE','NOMBRE'], as_index= False).count()
b

,PARALELO,CLASE,UNIDAD,BLOQUE,NOMBRE
0,1,Teórico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,7B,EMPRENDIMIENTO E INNOVACIÓN
1,1,Teórico,Facultad de Ciencias Naturales y Matemáticas,9M,CIENCIAS DE LA SOSTENIBILIDAD
2,1,Teórico,Facultad de Ciencias Naturales y Matemáticas,9M,CÁLCULO DE UNA VARIABLE
3,1,Teórico,Facultad de Ciencias Naturales y Matemáticas,9M,CÁLCULO VECTORIAL
4,1,Teórico,Facultad de Ciencias Naturales y Matemáticas,9M,ESTADÍSTICA
...,...,...,...,...,...
59,103,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11A,SISTEMAS DIGITALES I
60,103,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11C,PRINCIPIOS DE ELECTRÓNICA
61,104,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS IV
62,104,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS V


In [ ]:
s=pd.DataFrame(b[['NOMBRE']].value_counts()) 
s=s.reset_index()
s.columns=['Materias' ,'CantParalelos']
s

,Materias,CantParalelos
0,INGLÉS V,10
1,INGLÉS IV,8
2,INGLÉS II,6
3,INGLÉS III,6
4,SISTEMAS DIGITALES I,4
5,EMPRENDIMIENTO E INNOVACIÓN,4
6,PRINCIPIOS DE ELECTRÓNICA,4
7,INGLÉS I,4
8,TÉCNICAS DIETÉTICAS,3
9,PRODUCCIÓN VEGETAL I,2


In [ ]:
fig = px.line(s, x='Materias', y="CantParalelos",markers = True,text="CantParalelos",title='Cantidad')
fig.update_traces(textposition = "top center")
fig.show()  

## Capacidad por hora

In [ ]:
df_H_p

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION,COURSE,CLASE,lat,lon
0,2023,0S,miércoles,09:00:00,11:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico,-2.15230,-79.95582
1,2023,0S,lunes,09:00:00,12:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
2,2023,0S,miércoles,14:00:00,16:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico,-2.15230,-79.95582
3,2023,0S,lunes,14:00:00,17:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
4,2023,0S,lunes,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,03:00:00,4,Práctico,-2.14715,-79.96778
123,2023,0S,martes,15:30:00,17:00:00,INGLÉS IV,IDIG1009,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico,-2.14715,-79.96778
124,2023,0S,martes,15:30:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico,-2.14715,-79.96778
125,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,105,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO,03:00:00,5,Práctico,-2.14715,-79.96778


In [ ]:
df_Horario

,NOMBRE,HORARIO,INDICAR,PARALELO,DIA,COURSE,CLASE,size
1,PRODUCCIÓN VEGETAL I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
3,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,14:00:00,ENTRADA,1,lunes,1,Teórico,10
4,TÉCNICAS DIETÉTICAS,09:00:00,ENTRADA,1,lunes,1,Teórico,10
10,SISTEMAS DIGITALES I,09:00:00,ENTRADA,1,lunes,1,Teórico,10
13,PRINCIPIOS DE ELECTRÓNICA,09:00:00,ENTRADA,1,lunes,1,Teórico,10
...,...,...,...,...,...,...,...,...
207,CÁLCULO VECTORIAL,12:00:00,SALIDA,101,jueves,1,Práctico,10
221,PRINCIPIOS DE ELECTRÓNICA,11:00:00,SALIDA,102,jueves,1,Práctico,10
222,SISTEMAS DIGITALES I,12:00:00,SALIDA,102,jueves,1,Práctico,10
234,PRINCIPIOS DE ELECTRÓNICA,12:00:00,SALIDA,103,jueves,1,Práctico,10


In [ ]:
HI = df_H_p.groupby('HORAINICIO',as_index=False)['CAPACIDAD'].sum()
HF = df_H_p.groupby('HORAFIN',as_index=False)['CAPACIDAD'].sum()
HI,HF

(  HORAINICIO  CAPACIDAD
 0   09:00:00       1078
 1   09:30:00         30
 2   10:00:00        165
 3   11:00:00         80
 4   12:00:00         10
 5   14:00:00       1555
 6   15:30:00        510,
     HORAFIN  CAPACIDAD
 0  10:00:00        175
 1  11:00:00        210
 2  11:30:00         90
 3  12:00:00        818
 4  13:00:00         70
 5  15:30:00        510
 6  16:00:00         10
 7  17:00:00       1545)

In [ ]:
h_inicial= dt.datetime(2023, 1, 1, 9, 0, 0)
h_final=dt.datetime(2023, 1, 1, 17, 1, 0)
hora=dt.timedelta(hours=1)
time_1Hour =[(str(x).split('T')[1]).split('.')[0] for x in np.arange(h_inicial,h_final,hora)]
time_1Hour

['09:00:00',
 '10:00:00',
 '11:00:00',
 '12:00:00',
 '13:00:00',
 '14:00:00',
 '15:00:00',
 '16:00:00',
 '17:00:00']

In [ ]:
def capacidad_porhora(df,name_column,name_capacidad):
    i=0
    historia=0
    for hi in df[name_column]:
        valida= str(hi).split(':')[1]
        if valida == '00':
            df.loc[i,name_capacidad]+=historia
            historia=0
        else:
            historia = df.loc[i,name_capacidad]
            df.drop(i,axis=0,inplace=True)
        i+=1
    return df
HI,HF = capacidad_porhora(HI,'HORAINICIO','CAPACIDAD').reset_index(drop=True),capacidad_porhora(HF,'HORAFIN','CAPACIDAD').reset_index(drop=True)
HI.HORAINICIO=HI.HORAINICIO.astype(str)
HF.HORAFIN=HF.HORAFIN.astype(str)

In [ ]:
for hora in time_1Hour:
    if hora not in list(HI['HORAINICIO']):
        HI.loc[len(HI)]=[hora,0]
    if hora not in list(HF['HORAFIN']):
        HF.loc[len(HF)]=[hora,0]



In [ ]:
HI.HORAINICIO= pd.Categorical(HI.HORAINICIO,time_1Hour)
HI.HORAFIN=pd.Categorical(HF.HORAFIN,time_1Hour)
HI=HI.sort_values('HORAINICIO').reset_index(drop=True)
HF=HF.sort_values('HORAFIN').reset_index(drop=True)


/var/folders/pf/565wgnj55rl181qjmrmqq69m0000gn/T/ipykernel_15262/543690760.py:2: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [ ]:
df_capacidad = pd.DataFrame(index=time_1Hour,columns=['ENTRADA','SALIDA'])
df_capacidad['ENTRADA']=HI['CAPACIDAD'].values
df_capacidad['SALIDA']=HF['CAPACIDAD'].values

In [ ]:
columns = list(df_capacidad.columns)

In [ ]:
import plotly.express as px
fig10 = px.line(df_capacidad, x=df_capacidad.index, y=columns , title='FLUJOS DE ESTUDIANTES ', labels={'index':'HORARIOS','value':'NUMERO DE ESTUDIANTES'})
fig10.update_traces(textposition = "top center")
fig10.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title='TIEMPOS', orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ))
fig10.show()

In [ ]:
df7 = df_H_p.groupby(["UNIDAD","DIA","PARALELO","HORAINICIO","HORAFIN"],as_index=False)['CUPOPLANIFICADO'].sum()
df7.value_counts()

UNIDAD                                                DIA        PARALELO  HORAINICIO  HORAFIN   CUPOPLANIFICADO
ESCUELA SUPERIOR POLITECNICA DEL LITORAL              lunes      1         09:00:00    12:00:00  38                 1
                                                                 2         09:00:00    12:00:00  38                 1
Facultad de Ciencias Sociales y Humanísticas          miércoles  101       14:00:00    17:00:00  150                1
                                                                 102       14:00:00    17:00:00  120                1
                                                                 103       14:00:00    17:00:00  150                1
                                                                                                                   ..
                                                      martes     3         14:00:00    15:30:00  150                1
                                                             

In [ ]:
df_H_p.query("UNIDAD == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL' and DIA =='lunes' ")

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION,COURSE,CLASE,lat,lon
40,2023,0S,lunes,09:00:00,12:00:00,EMPRENDIMIENTO E INNOVACIÓN,ADMG1005,1,N,38,38,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,teóricoPractica,NAME017,EMAIL017,A101,7B,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.14704,-79.96609
50,2023,0S,lunes,09:00:00,12:00:00,EMPRENDIMIENTO E INNOVACIÓN,ADMG1005,2,N,38,38,PR,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,teóricoPractica,NAME022,EMAIL022,A101*,13B,CAMPUS GUSTAVO GALINDO,03:00:00,2,Teórico,-2.14558,-79.96507


In [ ]:
df_H_p.groupby(['UNIDAD'],as_index=False)[['HORAINICIO','HORAFIN','CUPOPLANIFICADO']].value_counts()

,UNIDAD,HORAINICIO,HORAFIN,CUPOPLANIFICADO,count
0,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,09:00:00,12:00:00,38,6
1,Facultad de Ciencias Naturales y Matemáticas,09:00:00,12:00:00,30,10
2,Facultad de Ciencias Naturales y Matemáticas,10:00:00,12:00:00,30,4
3,Facultad de Ciencias Naturales y Matemáticas,09:00:00,10:00:00,30,4
4,Facultad de Ciencias Naturales y Matemáticas,09:00:00,11:30:00,31,2
5,Facultad de Ciencias Naturales y Matemáticas,09:30:00,11:30:00,31,1
6,Facultad de Ciencias Sociales y Humanísticas,14:00:00,17:00:00,30,34
7,Facultad de Ciencias Sociales y Humanísticas,14:00:00,15:30:00,30,17
8,Facultad de Ciencias Sociales y Humanísticas,15:30:00,17:00:00,30,17
9,Facultad de Ciencias de la Vida,14:00:00,16:00:00,5,2


## Duracion


In [ ]:
dura =df_H_p[["PARALELO", 'CUPOPLANIFICADO' ,'CLASE','UNIDAD','BLOQUE' ,'NOMBRE', 'DIA','DURACION']].groupby(["PARALELO",'CUPOPLANIFICADO' ,'CLASE','UNIDAD','BLOQUE' ,'NOMBRE', 'DIA','DURACION'], as_index= False).count()
dura

,PARALELO,CUPOPLANIFICADO,CLASE,UNIDAD,BLOQUE,NOMBRE,DIA,DURACION
0,1,5,Teórico,Facultad de Ciencias de la Vida,3H,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,lunes,03:00:00
1,1,5,Teórico,Facultad de Ciencias de la Vida,3H,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,miércoles,02:00:00
2,1,5,Teórico,Facultad de Ciencias de la Vida,3H,PRODUCCIÓN VEGETAL I,lunes,03:00:00
3,1,5,Teórico,Facultad de Ciencias de la Vida,3H,PRODUCCIÓN VEGETAL I,miércoles,02:00:00
4,1,20,Teórico,Facultad de Ciencias de la Vida,3H,TÉCNICAS DIETÉTICAS,lunes,03:00:00
...,...,...,...,...,...,...,...,...
122,104,30,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS IV,miércoles,03:00:00
123,104,30,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS V,martes,01:30:00
124,104,30,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS V,miércoles,03:00:00
125,105,30,Práctico,Facultad de Ciencias Sociales y Humanísticas,9F,INGLÉS V,martes,01:30:00


In [ ]:
dura = dura.sort_values('DURACION')

In [ ]:
dura

,PARALELO,CUPOPLANIFICADO,CLASE,UNIDAD,BLOQUE,NOMBRE,DIA,DURACION
107,103,10,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11A,SISTEMAS DIGITALES I,jueves,01:00:00
39,1,35,Teórico,Facultad de Ingeniería en Electricidad y Compu...,11A,FUNDAMENTOS DE PROGRAMACIÓN,miércoles,01:00:00
74,101,10,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11C,PRINCIPIOS DE ELECTRÓNICA,jueves,01:00:00
72,101,10,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11A,SISTEMAS DIGITALES I,jueves,01:00:00
109,103,10,Práctico,Facultad de Ingeniería en Electricidad y Compu...,11C,PRINCIPIOS DE ELECTRÓNICA,jueves,01:00:00
...,...,...,...,...,...,...,...,...
46,2,30,Teórico,Facultad de Ciencias Sociales y Humanísticas,8F,INGLÉS II,lunes,03:00:00
44,2,30,Teórico,Facultad de Ciencias Sociales y Humanísticas,8E,INGLÉS IV,lunes,03:00:00
42,2,30,Teórico,Facultad de Ciencias Sociales y Humanísticas,8E,INGLÉS III,lunes,03:00:00
85,101,30,Práctico,Facultad de Ciencias Sociales y Humanísticas,8E,INGLÉS IV,miércoles,03:00:00


In [ ]:
unidades = list(dura.UNIDAD.unique())
for unid in unidades:
    df_t = dura[dura.UNIDAD == unid]
    df_t
    if list(df_t['UNIDAD'])[0] == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL':
        df_t.loc['200'] = [1, 'Teórico', '0','ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
        fig_dura = px.bar(df_t, x="DURACION", y="NOMBRE", color="NOMBRE", title=unid, height=800, labels={'NOMBRE': 'MATERIAS'}, facet_col='DIA',  facet_col_wrap=1, hover_data=['DURACION', 'PARALELO', 'CLASE','CUPOPLANIFICADO'], barmode='group')
        fig_dura.update_layout(
            font_family="Rockwell"
        )
        fig_dura.update_yaxes(
          showgrid=False)
        fig_dura.show()
    else:
        df_t.loc['200'] = [1, 'Teórico', '0','ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
        fig_dura = px.bar(df_t, x="DURACION", y="NOMBRE", color="NOMBRE", title=unid, height=800, facet_col='DIA', labels={'NOMBRE': 'MATERIAS'}, facet_col_wrap=1, hover_data=['DURACION', 'PARALELO', 'CLASE', 'CUPOPLANIFICADO'], barmode='group')
        fig_dura.update_layout(
            font_family="Rockwell"
        )
        fig_dura.update_yaxes(
          showgrid=False)
        fig_dura.show()
   

In [ ]:
dura.query("NOMBRE == 'EMPRENDIMIENTO E INNOVACIÓN'")

,PARALELO,CUPOPLANIFICADO,CLASE,UNIDAD,BLOQUE,NOMBRE,DIA,DURACION
93,101,38,Práctico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,7B,EMPRENDIMIENTO E INNOVACIÓN,miércoles,03:00:00
106,102,38,Práctico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,13B,EMPRENDIMIENTO E INNOVACIÓN,miércoles,03:00:00
40,1,38,Teórico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,7B,EMPRENDIMIENTO E INNOVACIÓN,lunes,03:00:00
41,1,38,Teórico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,7B,EMPRENDIMIENTO E INNOVACIÓN,martes,03:00:00
51,2,38,Teórico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,13B,EMPRENDIMIENTO E INNOVACIÓN,martes,03:00:00
50,2,38,Teórico,ESCUELA SUPERIOR POLITECNICA DEL LITORAL,13B,EMPRENDIMIENTO E INNOVACIÓN,lunes,03:00:00


In [ ]:
'''df_t1 = dura[dura.UNIDAD == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL']
df_t1.loc['200'] = [1, 'Teórico', 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
df_t1'''

"df_t1 = dura[dura.UNIDAD == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL']\ndf_t1.loc['200'] = [1, 'Teórico', 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']\ndf_t1"

## IRREGULARIDADES


In [ ]:
df_H_p.query("NOMBRE == 'INGLÉS I'")
## No exite paralelo 2 a diferencia de lo que esta en el academico

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION,COURSE,CLASE,lat,lon
26,2023,0S,martes,14:00:00,15:30:00,INGLÉS I,IDIG1006,1,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME012,EMAIL012,A001,8F,CAMPUS GUSTAVO GALINDO,01:30:00,1,Teórico,-2.14878,-79.96752
31,2023,0S,lunes,14:00:00,17:00:00,INGLÉS I,IDIG1006,1,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME012,EMAIL012,A001,8F,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.14878,-79.96752
53,2023,0S,martes,14:00:00,15:30:00,INGLÉS I,IDIG1006,3,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME024,EMAIL024,A003,8F,CAMPUS GUSTAVO GALINDO,01:30:00,3,Teórico,-2.14878,-79.96752
58,2023,0S,lunes,14:00:00,17:00:00,INGLÉS I,IDIG1006,3,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME024,EMAIL024,A003,8F,CAMPUS GUSTAVO GALINDO,03:00:00,3,Teórico,-2.14878,-79.96752
84,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS I,IDIG1006,101,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME012,EMAIL012,A001,8F,CAMPUS GUSTAVO GALINDO,03:00:00,1,Práctico,-2.14878,-79.96752
89,2023,0S,martes,15:30:00,17:00:00,INGLÉS I,IDIG1006,101,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME012,EMAIL012,A001,8F,CAMPUS GUSTAVO GALINDO,01:30:00,1,Práctico,-2.14878,-79.96752
112,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS I,IDIG1006,103,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME024,EMAIL024,A003,8F,CAMPUS GUSTAVO GALINDO,03:00:00,3,Práctico,-2.14878,-79.96752
117,2023,0S,martes,15:30:00,17:00:00,INGLÉS I,IDIG1006,103,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME024,EMAIL024,A003,8F,CAMPUS GUSTAVO GALINDO,01:30:00,3,Práctico,-2.14878,-79.96752


In [ ]:
df_H_p.NOMBRE.unique()

array(['PRODUCCIÓN VEGETAL I', 'DIAGNÓSTICO Y PROTECCIÓN VEGETAL I',
       'TÉCNICAS DIETÉTICAS', 'CÁLCULO DE UNA VARIABLE',
       'CÁLCULO VECTORIAL', 'SISTEMAS DIGITALES I',
       'PRINCIPIOS DE ELECTRÓNICA', 'ÁLGEBRA LINEAL', 'ESTADÍSTICA',
       'INGLÉS III', 'INGLÉS IV', 'INGLÉS I', 'INGLÉS II', 'INGLÉS V',
       'CIENCIAS DE LA SOSTENIBILIDAD', 'FUNDAMENTOS DE PROGRAMACIÓN',
       'EMPRENDIMIENTO E INNOVACIÓN'], dtype=object)

In [ ]:
#Los cupos en emprendimiento e innovación tienen inconsistencias con respecto al data set entregado

## FLEUNCIA ESTIMADA DE ESTUDIANTES RECIBIENDO CLASES POR FACULTAD

Nos permite ver como es el flujo de estudiantes que están recibiendo clases por facultad y como esto va cambiando durante el dia.
Para este proceso primero eliminamos las filas repetidas . 


In [ ]:
df_H_p.drop_duplicates()

,ANIO,TERMINO,DIA,HORAINICIO,HORAFIN,NOMBRE,CODIGOMATERIA,PARALELO,ESQUINCENA,CAPACIDAD,CUPOPLANIFICADO,MODALIDAD,UNIDAD,TIPOMATERIA,DOCENTE,EMAIL,AULA,BLOQUE,CAMPUS,DURACION,COURSE,CLASE,lat,lon
0,2023,0S,miércoles,09:00:00,11:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico,-2.15230,-79.95582
1,2023,0S,lunes,09:00:00,12:00:00,PRODUCCIÓN VEGETAL I,AGRG1032,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME001,EMAIL001,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
2,2023,0S,miércoles,14:00:00,16:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,02:00:00,1,Teórico,-2.15230,-79.95582
3,2023,0S,lunes,14:00:00,17:00:00,DIAGNÓSTICO Y PROTECCIÓN VEGETAL I,AGRG1025,1,N,5,5,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME002,EMAIL002,A106,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
4,2023,0S,lunes,09:00:00,12:00:00,TÉCNICAS DIETÉTICAS,NUTG2042,1,N,20,20,PR,Facultad de Ciencias de la Vida,teóricoPractica,NAME003,EMAIL003,A101,3H,CAMPUS GUSTAVO GALINDO,03:00:00,1,Teórico,-2.15230,-79.95582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,03:00:00,4,Práctico,-2.14715,-79.96778
123,2023,0S,martes,15:30:00,17:00:00,INGLÉS IV,IDIG1009,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME028,EMAIL028,A005,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico,-2.14715,-79.96778
124,2023,0S,martes,15:30:00,17:00:00,INGLÉS V,IDIG1010,104,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME029,EMAIL029,A104,9F,CAMPUS GUSTAVO GALINDO,01:30:00,4,Práctico,-2.14715,-79.96778
125,2023,0S,miércoles,14:00:00,17:00:00,INGLÉS V,IDIG1010,105,N,30,30,PR,Facultad de Ciencias Sociales y Humanísticas,teóricoPractica,NAME030,EMAIL030,A105,9F,CAMPUS GUSTAVO GALINDO,03:00:00,5,Práctico,-2.14715,-79.96778
